In [1]:
import igl

In [11]:
#https://github.com/libigl/libigl-tutorial-data

In [37]:
import numpy as np
import igl
import meshplot as mp
from scipy.spatial.transform import Rotation
import ipywidgets as iw
import time
from math import exp
import quaternion

In [13]:
V, F = igl.read_triangle_mesh('data/arm.obj')
C,BE,_,_,_,_ = igl.read_tgf('data/arm.tgf')
W = igl.read_dmat('data/arm-weights.dmat')
# labels = np.load('data/hand.label.npy').astype(int)
# v -= v.min(axis=0)
# v /= v.max()

In [14]:
# TV = np.concatenate((V, C))
# TE = BE + V.shape[0]

In [15]:
# labels = np.concatenate((np.zeros(V.shape[0]), np.array(range(1,C.shape[0]+1)))).astype(int)
labels = np.array(range(BE.shape[0])).astype(int)

In [112]:
quaternion.as_rotation_matrix(quaternion.as_quat_array([1,0,0,.5]))

array([[ 0.6, -0.8,  0. ],
       [ 0.8,  0.6,  0. ],
       [ 0. ,  0. ,  1. ]])

In [212]:
def circle_sum(q1, q2):
    return q1 + q2 if np.dot(quaternion.as_float_array(q1), quaternion.as_float_array(q2)) >= 0 else q1 - q2

def apply_weighted_rotation(vertices, wgts, quat, CoR, translation): #vertices, weights, rot (as quat), CoR, translation array 
    vnew = vertices.copy()
    for i in range(vertices.shape[0]):
        vi = vertices[i]
        wi = wgts[i]
        
        q = quaternion.as_quat_array([0,0,0,0])
        for j in range(BE.shape[0]): # #BE
            qj = quat[j]
            wij = wi[j]
            q = circle_sum(q, wij*qj)

        # wqi  = np.sum(qi, axis=0)
        q /= np.linalg.norm(quaternion.as_float_array(q))
        
        R = quaternion.as_rotation_matrix(q)
        
        Rtilda = np.zeros((3,3))
        ttilda = np.zeros((3,1))
        
        for j in range(BE.shape[0]):
            Rj = quaternion.as_rotation_matrix(quat[j]) #3 x 3
            tj = translation[j] # 1 x 3
            wij = wi[j]
            Rtilda += wij*Rj
            ttilda += wij*tj.reshape((3,1))
            
        t = Rtilda @ CoR[i].reshape((3,1)) + ttilda - R @ CoR[i].reshape((3,1))
        
        vnew[i] = (R @ vi.reshape((3,1)) + t).reshape(3)
            

        # vi_q = quaternion.as_quat_array(np.concatenate(([0], vi)))

        # qi_q = quaternion.as_quat_array(wqi)

        # res = qi_q.inverse() * vi_q * qi_q

        # vnew[i] = [res.x, res.y, res.z]

    return vnew

def apply_weighted_rotation_linear(rot, vertices): 
    vertices_new = np.zeros(vertices.shape)
    for i in range(vertices.shape[0]):
        vi = vertices[i,:]
        wi = W[i,:]
        

       
        # wqi /= np.linalg.norm(wqi)
        
        r = Rotation.from_quat(rot)
        
        rotmat = r.as_matrix() #4 by 3 x 3
        
        for j in range(rot.shape[0]):
            wj = wi[j]
            Tj = rotmat[j,:]
            
            qi = np.eye(Tj.shape[0]) * wj @ Tj
            wqi  = qi @ vi
            vertices_new[i] += wqi
        
    return vertices_new

def apply_rotation(rot, vertices):
    vertices_new = vertices.copy()
    for i in range(vertices.shape[0]):
        vi = vertices[i,:]
        # wi = W[i,:]
        # qi = np.diag(wi) @ rot

        # wqi  = np.sum(qi, axis=0)
        # wqi /= np.linalg.norm(wqi)

        vi_q = quaternion.as_quat_array(np.concatenate(([0], vi)))

        # qi_q = quaternion.as_quat_array(wqi)
        qi_q = quaternion.as_quat_array(rot)

        # res = qi_q * vi_q * qi_q.inverse()
        res = qi_q.inverse() * vi_q * qi_q

        vertices_new[i] = [res.x, res.y, res.z]

        # print(ri.shape)
    return vertices_new

In [213]:
def similarity(Wp, Wv, sigma=1):
    #BE shape
    Wp = Wp if len(Wp.shape) == 1 else Wp.reshape(-1)
    Wv = Wv if len(Wv.shape) == 1 else Wv.reshape(-1)
    
    tot = 0
    for j in range(Wp.shape[0]):
        for k in range(j+1, Wv.shape[0]):
            tot += Wp[j]*Wp[k]*Wv[j]*Wv[k]*exp( -(Wp[j]*Wv[k] - Wp[k]*Wv[j])**2 / sigma**2)
    return tot

def CoR(i, weights, vertices, faces):
    
    num = np.zeros([1, 3])
    denom = np.zeros([1,3])
    for t in range(faces.shape[0]):
        s = similarity(weights[i], (weights[faces[t,0]] + weights[faces[t,1]] + weights[faces[t,2]]) / 3)
        v = (vertices[faces[t,0]] + vertices[faces[t,1]] + vertices[faces[t,2]]) / 3
        a = igl.doublearea(vertices, faces[[t]]) / 2 

        num += s * v * a
        denom += s * a
    
    pi = num / denom

    return pi

In [214]:
# pb = iw.IntProgress(
#     value=0,
#     min=0,
#     max=V.shape[0],
#     description='Loading:',
#     bar_style='', # 'success', 'info', 'warning', 'danger' or ''
#     style={'bar_color': 'maroon'},
#     orientation='horizontal'
# )
# from IPython.display import display

# import time
# display(pb)

# P = np.zeros(V.shape)
# for i in range(P.shape[0]):
#     P[i] = CoR(i, W, V, F)
#     # time.sleep(.1)
#     pb.value = i

In [215]:
# import pickle
# pickle.dump(P, open("data/CoR.p", "wb"))

In [216]:
P = pickle.load(open('data/CoR.p', 'rb'))

In [217]:
handle_vertex_positions = V.copy()

pos_f_saver = np.zeros((labels.max()+1, 7))

def pos_f(s,x,y,z, w, α, β, γ):
    print(np.isin(labels, s))
    clicked_edges = BE[np.isin(labels, s)]
    slices = []
    for e in clicked_edges:
        for v in e:
            if v not in slices:
                slices.append(v)
                
    print(slices)
 
        
    # r = Rotation.from_euler('xyz', [α, β, γ], degrees=True)
    # r = quaternion.from_quat([w, α, β, γ])
    r = np.array([w, α, β, γ])
    # v_slice = TV[slices] + np.array([[x,y,z]])
    v_slice = C[slices] + np.array([[x,y,z]])
    
    # print(r.as_quat())
    
    # center = v_slice.mean(axis=0)
    C1 = C.copy()
    # C1[slices] = r.apply(v_slice)
    C1[slices] = apply_rotation(r, v_slice)
    for si in s:
        print("si " ,si)
        pos_f_saver[si] = [x,y,z,w,α,β,γ]
    
    # Qt0  = quaternion.as_quat_array(igl.directed_edge_orientations(C,BE))
    # Qt1  =  quaternion.as_quat_array(igl.directed_edge_orientations(C1,BE))
    
    # qto * diff = qt1
    # diff = qt0^-1 * qt1
    print(BE)
    
    q = np.zeros((BE.shape[0], 4))
    t = np.zeros((BE.shape[0], 3))
    for i in range(BE.shape[0]):
        # res =  Qt0[i].inverse() * Qt1[i] 
        # diff[i,:] = [res.w, res.x, res.y, res.z]
        xi,yi,zi,wi,αi,βi,γi = pos_f_saver[i]
        q[i,:] = [wi,αi,βi,γi]
        t[i,:] = [xi, yi, zi]
    
    q = quaternion.as_quat_array(q)
    print(q)
    print(t)
    
    handle_vertex_positions = V.copy()
    # handle_vertex_positions = apply_weighted_rotation(diff, V) 
    handle_vertex_positions = apply_weighted_rotation(V,W, q, P, t)
    print(handle_vertex_positions)
    

    v_deformed = pos_f.deformer(handle_vertex_positions)
    # p.update_object(vertices = v_deformed)
    p.update_object(vertices = v_deformed)
    p.remove_object(max(p._Viewer__objects.keys()))
    p.add_edges(C1+np.repeat([[0,.25,0]], C1.shape[0], axis=0), BE, shading={"line_color": "green"})
  
pos_f.deformer = lambda x:x

In [218]:
def widgets_wrapper():
    # segment_widget = iw.Dropdown(options=np.arange(labels.max()) + 1)
    segment_widget = iw.SelectMultiple(options=np.arange(labels.max()+1))
    translate_widget = {i:iw.FloatSlider(min=-1, max=1, value=0) 
                        for i in 'xyz'}
    # rotate_widget = {a:iw.FloatSlider(min=-90, max=90, value=0, step=1) 
    #                  for a in 'αβγ'}
    real_widget = {a:iw.FloatSlider(min=-2, max=2, value=1, step=.25) 
                     for a in 'w'}
    imag_widget = {a:iw.FloatSlider(min=-1, max=1, value=0, step=.1) 
                     for a in 'αβγ'}

    def update_seg(*args):
        (translate_widget['x'].value,translate_widget['y'].value,
        translate_widget['z'].value, real_widget['w'].value,
        imag_widget['α'].value,imag_widget['β'].value,
        imag_widget['γ'].value) = pos_f_saver[segment_widget.value]
    segment_widget.observe(update_seg, 'value')
    widgets_dict = dict(s=segment_widget)
    widgets_dict.update(translate_widget)
    widgets_dict.update(real_widget)
    widgets_dict.update(imag_widget)
    return widgets_dict

In [219]:
def position_deformer(target_pos):
    '''Fill in this function to change positions'''
    return target_pos
''' (Optional) Register this function to perform interactive deformation
pos_f.deformer = position_deformer
'''

' (Optional) Register this function to perform interactive deformation\npos_f.deformer = position_deformer\n'

In [220]:

## Widget UI

p = mp.plot(V, F)
# p = mp.plot(v,f)
# p.add_points(VE)
p.add_edges(C,BE, shading={"line_color": "green"});
iw.interact(pos_f,
            **widgets_wrapper())

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0001610…

interactive(children=(SelectMultiple(description='s', options=(0, 1, 2, 3), value=()), FloatSlider(value=0.0, …

<function __main__.pos_f(s, x, y, z, w, α, β, γ)>

In [210]:
A = np.array([[1,1,1], [1,1,1], [1,1,1]]).astype("float64")
BA = np.array([[0,2], [2,1], [1,0]])

In [211]:
r1 = igl.directed_edge_orientations(A, BA)

In [556]:
x = A.copy()
x[1,:] += 20
print(x)

[[ 1.  1.  1.]
 [21. 21. 21.]
 [ 1.  1.  1.]]


In [557]:
r2 = igl.directed_edge_orientations(x, BA)
r2

array([[ 0.        ,  0.        ,  0.        ,  0.70710678],
       [ 0.        , -0.32505758,  0.32505758,  0.88807383],
       [ 0.        ,  0.62796303, -0.62796303,  0.45970084]])

In [558]:
# W is expressed in terms of bones
# So we need to determine rotation of each bone then multiply by weights and sum to get new vertices
# As we move the vertices up down, rotate we have to recalculate the bone rotations and then above

In [ ]:
# // vQ is a list of rotations as quaternions
# // vT is a list of translations
# igl::dqs(V,W,vQ,vT,U);

In [75]:
# VE,EE,P,BE,CE,PE
Q = igl.directed_edge_orientations(C,BE);
# Q #E list of quaternions

In [356]:
x = []
x.extend([3,4])

In [357]:
x

[3, 4]

In [795]:
np.all(np.array([0,0,0,0]) == 0)

True

In [152]:
# TQ = np.repeat([[1,0,0,0]], 4, axis = 0)
# TQ = np.array(TQ)
TQ = np.array([[0,0,0,1],
               [0,0,0,1],
               [0,0,0,1],
               [0,0,0,1]])
TQ

array([[0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1]])

In [167]:
quaternion.as_quat_array(TQ)[0] / 4

quaternion(0, 0, 0, 0.25)

In [469]:
Vprime = V.copy()
Vprime = apply_rotation(TQ, Vprime)
p = mp.plot(Vprime, F)
p.add_mesh(V+.5, F)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.000161…

1

In [470]:
np.sum(Vprime - V)

-3450.1874185494

In [85]:
np.sum(x,axis=1)

array([4, 8])

In [81]:
y = np.array([1, 0,0,.5])
x = np.array([-5, 0,0,.5])

In [146]:
def circle_sum(q1, q2):
    return q1 + q2 if np.dot(quaternion.as_float_array(q1), quaternion.as_float_array(q2)) >= 0 else q1 - q2

In [94]:
A = quaternion.as_quat_array(y)
B = quaternion.as_quat_array(x)

In [168]:
np.dot(quaternion.as_float_array(A), quaternion.as_float_array(B))

-4.75